<a href="https://colab.research.google.com/github/Zernach/BetterReads-DS/blob/master/How_to_Generate_Book_Recommendations_KNearestNeighbors_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
**KNN model of 10k dataset**


_using data found on kaggle from Goodreads_


_books.csv contains information for 10,000 books, such as ISBN, authors, title, year_


_ratings.csv is a collection of user ratings on these books, from 1 to 5 stars_

In [0]:
# imports


import numpy as pd
import pandas as pd
import pickle


from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import re

**Books dataset**

In [0]:
import psycopg2
engine = psycopg2.connect(
    database="betterreads",
    user="betterreadsadmin",
    password="betterreadsadmin",
    host="betterreads-datascience-database.cvmplnwee5ws.us-east-1.rds.amazonaws.com",
    port='5432')

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
db_books = pd.read_sql_query("SELECT * FROM gb_data;", engine)
db_books['original_title'] = db_books['title']
db_books

,googleid,title,authors,publisher,publisheddate,description,isbn,pagecount,categories,thumbnail,smallthumbnail,lang,webreaderlink,textsnippet,isebook,averagerating,maturityrating,ratingscount,subtitle,original_title
0,PZ1cAAAACAAJ,"Black Berry, Sweet Juice",['Lawrence Hill'],HarperFlamingoCanada,2001,None,0002000202,245.0,"['Authors, Canadian (English) 20th century Bio...",None,None,en,http://play.google.com/books/reader?id=PZ1cAAA...,"The son of a black father and a white mother, ...",False,3.5,NOT_MATURE,3.0,On Being Black and White in Canada,"Black Berry, Sweet Juice"
1,sWzxAAAAMAAJ,The taste of metal,['Jack Todd'],HarperFlamingo,2001-03-29,None,WISC:89077171486,253.0,['Fiction'],http://books.google.com/books/content?id=sWzxA...,http://books.google.com/books/content?id=sWzxA...,en,http://play.google.com/books/reader?id=sWzxAAA...,He explains exactly what I need to do to get l...,False,3.0,NOT_MATURE,1.0,a deserter's story,The taste of metal
2,TbOgAAAAMAAJ,Crete on the half shell,['Byron Ayanoglu'],HarperFlamingo,2003-02-20,None,IND:30000086930678,268.0,['Cooking'],http://books.google.com/books/content?id=TbOgA...,http://books.google.com/books/content?id=TbOgA...,en,http://play.google.com/books/reader?id=TbOgAAA...,For information address: HarperCollins Publish...,False,NaN,NOT_MATURE,NaN,"a story about an island, good friends and food",Crete on the half shell
3,B5A7xgEACAAJ,Gloria,['Keith Maillard'],Harpercollins Canada,1999,Light in the Company of Women glittered with t...,0006481752,665.0,['Fiction'],http://books.google.com/books/content?id=B5A7x...,http://books.google.com/books/content?id=B5A7x...,en,http://play.google.com/books/reader?id=B5A7xgE...,None,False,4.5,NOT_MATURE,2.0,None,Gloria
4,7Pd_QgAACAAJ,Honey and Ashes,['Janice Kulyk Keefer'],Harpercollins Canada,1998,"Janice Kulyk Keefer was born in Canada, a coun...",0002554437,338.0,['Biography & Autobiography'],http://books.google.com/books/content?id=7Pd_Q...,http://books.google.com/books/content?id=7Pd_Q...,en,http://play.google.com/books/reader?id=7Pd_QgA...,"&quot;As the child of immigrants, confronted b...",False,4.0,NOT_MATURE,1.0,A Story of Family,Honey and Ashes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183153,V1duDmCqlPEC,The Life of Helen Alone,['Karen Lawrence'],None,1987,"Helen trades her safe, single life for the per...",0345330420,293.0,['Fiction'],http://books.google.com/books/content?id=V1duD...,http://books.google.com/books/content?id=V1duD...,en,http://play.google.com/books/reader?id=V1duDmC...,"Helen trades her safe, single life for the per...",False,NaN,NOT_MATURE,NaN,None,The Life of Helen Alone
183154,zi1Es2L5MvUC,Normandie Triangle,['Justin Scott'],None,1982-11,When a Nazi saboteur engineers the sinking of ...,0345306406,576.0,['Fiction'],http://books.google.com/books/content?id=zi1Es...,http://books.google.com/books/content?id=zi1Es...,en,http://play.google.com/books/reader?id=zi1Es2L...,When a Nazi saboteur engineers the sinking of ...,False,NaN,NOT_MATURE,NaN,None,Normandie Triangle
183155,y7k6CQAAQBAJ,Missing Time,['Budd hopkins'],Mantesh,1988-03-12,The arrival of extraterrestrial visitors is on...,9780345353351,260.0,"['Body, Mind & Spirit']",http://books.google.com/books/content?id=y7k6C...,http://books.google.com/books/content?id=y7k6C...,en,http://play.google.com/books/reader?id=y7k6CQA...,But Hopkins could not have told the stories of...,False,4.5,NOT_MATURE,2.0,,Missing Time
183156,_0skthx0BLAC,Perdido Street Station,['China Miéville'],Bas-Lag,2001,"In squallid and brutal New Crobuzon, a Garuda ...",0345443020,710.0,['Fiction'],http://books.google.com/books/content?id=_0skt...,http://books.google.com/books/content?id=_0skt...,en,http://play.google.com/books/reader?id=_0skthx...,"In squallid and brutal New Crobuzon, a Garuda ...",False,4.5,NOT_MATURE,17.0,None,Perdido Street Station


In [0]:
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')

In [0]:
print(books.dtypes)
print(db_books.dtypes)

book_id                        int64
goodreads_book_id              int64
best_book_id                   int64
work_id                        int64
books_count                    int64
isbn                          object
isbn13                       float64
authors                       object
original_publication_year    float64
original_title                object
title                         object
language_code                 object
average_rating               float64
ratings_count                  int64
work_ratings_count             int64
work_text_reviews_count        int64
ratings_1                      int64
ratings_2                      int64
ratings_3                      int64
ratings_4                      int64
ratings_5                      int64
image_url                     object
small_image_url               object
dtype: object
googleid           object
title              object
authors            object
publisher          object
publisheddate      object
descr

In [0]:
merged_books = books.join(db_books, lsuffix='_google', rsuffix='_old')
merged_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn_google,isbn13,authors_google,original_publication_year,original_title_google,title_google,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,googleid,title_old,authors_old,publisher,publisheddate,description,isbn_old,pagecount,categories,thumbnail,smallthumbnail,lang,webreaderlink,textsnippet,isebook,averagerating,maturityrating,ratingscount,subtitle,original_title_old
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,PZ1cAAAACAAJ,"Black Berry, Sweet Juice",['Lawrence Hill'],HarperFlamingoCanada,2001,None,0002000202,245.0,"['Authors, Canadian (English) 20th century Bio...",None,None,en,http://play.google.com/books/reader?id=PZ1cAAA...,"The son of a black father and a white mother, ...",False,3.5,NOT_MATURE,3.0,On Being Black and White in Canada,"Black Berry, Sweet Juice"
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,sWzxAAAAMAAJ,The taste of metal,['Jack Todd'],HarperFlamingo,2001-03-29,None,WISC:89077171486,253.0,['Fiction'],http://books.google.com/books/content?id=sWzxA...,http://books.google.com/books/content?id=sWzxA...,en,http://play.google.com/books/reader?id=sWzxAAA...,He explains exactly what I need to do to get l...,False,3.0,NOT_MATURE,1.0,a deserter's story,The taste of metal
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,TbOgAAAAMAAJ,Crete on the half shell,['Byron Ayanoglu'],HarperFlamingo,2003-02-20,None,IND:30000086930678,268.0,['Cooking'],http://books.google.com/books/content?id=TbOgA...,http://books.google.com/books/content?id=TbOgA...,en,http://play.google.com/books/reader?id=TbOgAAA...,For information address: HarperCollins Publish...,False,NaN,NOT_MATURE,NaN,"a story about an island, good friends and food",Crete on the half shell
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,B5A7xgEACAAJ,Gloria,['Keith Maillard'],Harpercollins Canada,1999,Light in the Company of Women glittered with t...,0006481752,665.0,['Fiction'],http://books.google.com/books/content?id=B5A7x...,http://books.google.com/books/content?id=B5A7x...,en,http://play.google.com/books/reader?id=B5A7xgE...,None,False,4.5,NOT_MATURE,2.0,None,Gloria
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,7Pd_QgAACAAJ,Honey and Ashes,['Janice Kulyk Keefer'],Harpercollins Canada,1998,"Janice Kulyk Keefer was born in Canada, a coun...",0002554437,338.0,['Biography & Autobiography'],http://books.google.com/books/content?id=7Pd_Q...,http://books.google.com/books/content?id=7Pd_Q...,en,http://play.google.com/books/reader?id=7Pd_QgA...,"&quot;As the child of immigrants, confronted b...",False,4.0,NOT_MATURE,1.0,A Story of Family,Honey and Ashes


In [0]:
print(books.shape)
books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [0]:
books['title'].isnull().sum()

0

**Ratings dataset**

In [0]:
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

In [0]:
print(ratings.shape)
ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


**Trim down the data**

_In order to make a user rating matrix we will only need bood_id and title._



In [0]:
cols = ['book_id', 'title']

books = books[cols]
books.head()

,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,"Twilight (Twilight, #1)"
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**Clean up book titles**

_Book titles are messy, special characters, empty spaces, brackets clutter up the titles_

In [0]:
def clean_book_titles(title):
  title = re.sub(r'\([^)]*\)', '', title) # handles brackets
  title = re.sub(' + ', ' ', title) #compresses multi spaces into a single space
  title = title.strip() # handles special characters
  return title

In [0]:
books['title'] = books['title'].apply(clean_book_titles)

books.head()

,book_id,title
0,1,The Hunger Games
1,2,Harry Potter and the Sorcerer's Stone
2,3,Twilight
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**Create feature matrix**

_Combine datasets to get a new dataset of user ratings for each book_

In [0]:
books_ratings = pd.merge(ratings, books, on='book_id')

print(books_ratings.shape)

books_ratings.head()


(5976479, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Remove rows with same user_id and book title**



In [0]:
user_ratings = books_ratings.drop_duplicates(['user_id', 'title'])

print(user_ratings.shape)
user_ratings.head()

(5972713, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Pivot table to create user_ratings matrix**

_Each column is a user and each row is a book. The entries in the martix are the user's rating for that book._

In [0]:
user_matrix = user_ratings.pivot(index='title', columns='user_id', values='rating').fillna(0)

user_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,53385,53386,53387,53388,53389,53390,53391,53392,53393,53394,53395,53396,53397,53398,53399,53400,53401,53402,53403,53404,53405,53406,53407,53408,53409,53410,53411,53412,53413,53414,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#GIRLBOSS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
'Tis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
user_matrix.shape

(9800, 53424)

**Compress the matrix since it is extremely sparse**

_Whole lotta zeros_

_

In [0]:
compressed = csr_matrix(user_matrix.values)

In [0]:
# build and train knn

# unsupervised learning
# using cosine to measure space/distance

knn = NearestNeighbors(algorithm='brute', metric='cosine')

knn.fit(compressed)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
%%time

def get_recommendations(book_title, matrix=user_matrix, model=knn, topn=50):
    book_index = list(matrix.index).index(book_title)
    distances, indices = model.kneighbors(matrix.iloc[book_index,:].values.reshape(1,-1), n_neighbors=topn+1)
    print('Recommendations for {}:'.format(matrix.index[book_index]))
    for i in range(1, len(distances.flatten())):
        print('{}. {}, distance = {}'.format(i, matrix.index[indices.flatten()[i]], "%.3f"%distances.flatten()[i]))
    print()
    
get_recommendations("Harry Potter and the Sorcerer's Stone")
get_recommendations("Pride and Prejudice")
get_recommendations("Matilda")

Recommendations for Harry Potter and the Sorcerer's Stone:
1. Harry Potter and the Prisoner of Azkaban, distance = 0.320
2. Harry Potter and the Chamber of Secrets, distance = 0.327
3. Harry Potter and the Goblet of Fire, distance = 0.331
4. Harry Potter and the Order of the Phoenix, distance = 0.342
5. Harry Potter and the Half-Blood Prince, distance = 0.348
6. Harry Potter and the Deathly Hallows, distance = 0.349
7. The Hunger Games, distance = 0.410
8. The Hobbit, distance = 0.475
9. Twilight, distance = 0.477
10. The Fellowship of the Ring, distance = 0.507
11. To Kill a Mockingbird, distance = 0.507
12. The Lion, the Witch, and the Wardrobe, distance = 0.533
13. The Great Gatsby, distance = 0.558
14. Angels & Demons, distance = 0.559
15. Catching Fire, distance = 0.564
16. Pride and Prejudice, distance = 0.571
17. Mockingjay, distance = 0.583
18. The Golden Compass, distance = 0.586
19. 1984, distance = 0.586
20. Eragon, distance = 0.587
21. The Da Vinci Code, distance = 0.590
22

In [0]:
pickle.dump(knn, open('knn_model.pkl','wb'))
